In [1]:
# example to work with MPC. 

import os
%load_ext autoreload
%autoreload 2

from FlexivPy.robot.model.pinocchio import FlexivModel


urdf = "/home/FlexivPy/FlexivPy/assets/real_rizon_with_gripper.urdf"
flexiv_model =  FlexivModel(urdf = urdf, render=True)
flexiv_model.display(flexiv_model.q0)

# lets continue here!!

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [19]:
# lets get the end effector pose.
# small example




import numpy as np
import crocoddyl
import pinocchio
import time



from dataclasses import dataclass
@dataclass
class MPCconfig:
    gripperPose_run: float  = 1.
    gripperPose_terminal: float = 1.
    xReg : float = 1.
    uReg: float =  1e-1
    vel_terminal : float = 1e2






q0 = flexiv_model.q0
print("q0", q0)
dq = np.zeros(7)
D = flexiv_model.getInfo(q0, dq)
print(D["poses"].keys())
T0 = D["poses"]["link7"]
print(T0)

# we want
dx_des = [0, 0, -.2]
Tdes = np.copy(T0)
Tdes[:3, 3] += np.array(dx_des)

print("T0")
print(Tdes)
print(T0)
print(Tdes)

robot_model = flexiv_model.robot.model
state = crocoddyl.StateMultibody(robot_model)
actuation = crocoddyl.ActuationModelFull(state)
x0 = np.concatenate([q0, np.zeros(7)])

nu = 7

runningCostModel = crocoddyl.CostModelSum(state)
terminalCostModel = crocoddyl.CostModelSum(state)

# uResidual = crocoddyl.ResidualModelControlGrav(state, nu)
uResidual = crocoddyl.ResidualModelJointAcceleration(state, nu)
xResidual = crocoddyl.ResidualModelState(state, x0, nu)

framePlacementResidual = crocoddyl.ResidualModelFramePlacement(
    state,
    robot_model.getFrameId("link7"),
    pinocchio.SE3(Tdes),
    nu,
)

goalTrackingCost = crocoddyl.CostModelResidual(state, framePlacementResidual)
xRegCost = crocoddyl.CostModelResidual(state, xResidual)


xResidual2 = crocoddyl.ResidualModelState(state, x0, nu)

velRegCost = crocoddyl.CostModelResidual(
    state,
    crocoddyl.ActivationModelWeightedQuad(np.concatenate([np.zeros(7), np.ones(7)])),
    xResidual2,
)


uRegCost = crocoddyl.CostModelResidual(state, uResidual)


# Then let's added the running and terminal cost functions
runningCostModel.addCost("gripperPose", goalTrackingCost, 1)
runningCostModel.addCost("xReg", xRegCost, 1e0)
runningCostModel.addCost("uReg", uRegCost, 1e-1)
terminalCostModel.addCost("gripperPose", goalTrackingCost, 1e3)
terminalCostModel.addCost("velReg", velRegCost, 1e2)


# todo: we can also include the liner acceleration/velocity of the end effector!


# Next, we need to create an action model for running and terminal knots. The
# forward dynamics (computed using ABA) are implemented
# inside DifferentialActionModelFreeFwdDynamics.
dt = 0.05
runningModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(
        state, actuation, runningCostModel
    ),
    dt,
)
terminalModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(
        state, actuation, terminalCostModel
    ),
    0.0,
)

# For this optimal control problem, we define 100 knots (or running action
# models) plus a terminal knot
T = 20
problem = crocoddyl.ShootingProblem(x0, [runningModel] * T, terminalModel)

# Creating the DDP solver for this OC problem, defining a logger
solver = crocoddyl.SolverFDDP(problem)


solver.setCallbacks(
    [
        crocoddyl.CallbackVerbose(),
        crocoddyl.CallbackLogger(),
    ]
)

# Solving it with the solver algorithm
out = solver.solve()
# solution is in solver.xs
solver.us
time.sleep(0.01)  # clear prints
print(
    f"Initial T {T0}\n" f"Desired T {Tdes}\n",
    "Finally reached = ",
    solver.problem.terminalData.differential.multibody.pinocchio.oMf[
        robot_model.getFrameId("link7")
    ].translation.T,
)


log = solver.getCallbacks()[1]
crocoddyl.plotOCSolution(log.xs, log.us, figIndex=1, show=False)
crocoddyl.plotConvergence(
    log.costs, log.pregs, log.dregs, log.grads, log.stops, log.steps, figIndex=2
)

display = crocoddyl.MeshcatDisplay(flexiv_model.robot)

display.rate = -1
display.freq = 1
while True:
    print("displaying")
    display.displayFromSolver(solver)
    time.sleep(1.0)

q0 [ 0.    -0.698  0.     1.571 -0.     0.698 -0.   ]
dict_keys(['link1', 'link2', 'link3', 'link4', 'link5', 'link6', 'link7'])
[[ 9.99999979e-01  1.37562903e-16 -2.03694277e-04  7.01775245e-01]
 [ 1.37598041e-16 -1.00000000e+00  1.72486120e-16 -1.13000000e-01]
 [-2.03694277e-04 -1.72514145e-16 -9.99999979e-01  5.25812805e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
T0
[[ 9.99999979e-01  1.37562903e-16 -2.03694277e-04  7.01775245e-01]
 [ 1.37598041e-16 -1.00000000e+00  1.72486120e-16 -1.13000000e-01]
 [-2.03694277e-04 -1.72514145e-16 -9.99999979e-01  3.25812805e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.99999979e-01  1.37562903e-16 -2.03694277e-04  7.01775245e-01]
 [ 1.37598041e-16 -1.00000000e+00  1.72486120e-16 -1.13000000e-01]
 [-2.03694277e-04 -1.72514145e-16 -9.99999979e-01  5.25812805e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 9.99999979e-01  1.37562903e-16 -2.03694277e-04  7.01775245

KeyboardInterrupt: 

In [8]:
from dataclasses import dataclass
import numpy as np




import numpy as np
import crocoddyl
import pinocchio
import time


@dataclass
class Mpc_cfg:
    gripperPose_run: float = 1.0
    gripperPose_terminal: float = 1e3
    xReg: float = 1.0
    uReg: float = 1e-1
    vel_terminal: float = 1e2
    dt: float = 0.05
    horizon: int = 20


class Mpc_generator:
    def __init__(self, robot, x0, Tdes, mpc_cfg: Mpc_cfg = Mpc_cfg()):

        self.mpc_cfg = mpc_cfg
        self.robot = robot
        state = crocoddyl.StateMultibody(robot)
        actuation = crocoddyl.ActuationModelFull(state)
        x0 = np.concatenate([q0, np.zeros(7)])

        nu = 7

        runningCostModel = crocoddyl.CostModelSum(state)
        terminalCostModel = crocoddyl.CostModelSum(state)

        # uResidual = crocoddyl.ResidualModelControlGrav(state, nu)
        # uResidual = crocoddyl.ResidualModelJointEffort(state, nu)
        uResidual = crocoddyl.ResidualModelJointAcceleration(state, nu)
        xResidual = crocoddyl.ResidualModelState(state, x0, nu)

        framePlacementResidual = crocoddyl.ResidualModelFramePlacement(
            state,
            self.robot.getFrameId("link7"),
            pinocchio.SE3(Tdes),
            nu,
        )

        goalTrackingCost = crocoddyl.CostModelResidual(state, framePlacementResidual)
        xRegCost = crocoddyl.CostModelResidual(state, xResidual)

        velRegCost = crocoddyl.CostModelResidual(
            state,
            crocoddyl.ActivationModelWeightedQuad(
                np.concatenate([np.zeros(7), np.ones(7)])
            ),
            xResidual,
        )

        uRegCost = crocoddyl.CostModelResidual(state, uResidual)

        # Then let's added the running and terminal cost functions
        runningCostModel.addCost(
            "gripperPose", goalTrackingCost, mpc_cfg.gripperPose_run
        )
        runningCostModel.addCost("xReg", xRegCost, mpc_cfg.xReg)
        runningCostModel.addCost("uReg", uRegCost, mpc_cfg.uReg)
        terminalCostModel.addCost(
            "gripperPose", goalTrackingCost, mpc_cfg.gripperPose_terminal
        )
        terminalCostModel.addCost("velReg", velRegCost, mpc_cfg.vel_terminal)

        runningModel = crocoddyl.IntegratedActionModelEuler(
            crocoddyl.DifferentialActionModelFreeFwdDynamics(
                state, actuation, runningCostModel
            ),
            mpc_cfg.dt,
        )
        terminalModel = crocoddyl.IntegratedActionModelEuler(
            crocoddyl.DifferentialActionModelFreeFwdDynamics(
                state, actuation, terminalCostModel
            ),
            0.0,
        )

        self.problem = crocoddyl.ShootingProblem(
            x0, [runningModel] * mpc_cfg.horizon, terminalModel
        )
        self.solver = crocoddyl.SolverFDDP(self.problem)

        self.solver.setCallbacks(
            [
                crocoddyl.CallbackVerbose(),
                crocoddyl.CallbackLogger(),
            ]
        )

    def update_x0(self, x0):
        self.solver.problem.x0 = x0

        for k in range(self.solver.problem.T):
            self.solver.problem.runningModels[k].differential.costs.costs[
                "xReg"
            ].cost.residual.reference = x0

    def update_Tdes(self, Tdes):
        for k in range(self.solver.problem.T):
            self.solver.problem.runningModels[k].differential.costs.costs[
                "gripperPose"
            ].cost.residual.reference = Tdes
        self.problem.terminal_model.differential.costs.costs[
            "gripperPose"
        ].cost.residual.reference = Tdes


q0 = flexiv_model.q0
print("q0", q0)
dq = np.zeros(7)
D = flexiv_model.getInfo(q0, dq)
print(D["poses"].keys())
T0 = D["poses"]["link7"]
print(T0)

# we want
dx_des = [0, 0, -0.5]
Tdes = np.copy(T0)
Tdes[:3, 3] += np.array(dx_des)

mpc_cfg = Mpc_cfg()


mpc_generator = Mpc_generator(
    flexiv_model.robot.model, x0=flexiv_model.q0, Tdes=Tdes, mpc_cfg=mpc_cfg
)
mpc_generator.solver.solve()

# get the solution

# get the first control, and desired state

u = mpc_generator.solver.us[0]
x_des = mpc_generator.solver.xs[0]

print("sending to the robot!")

print("u", u)
print("x_des", x_des)


display = crocoddyl.MeshcatDisplay(flexiv_model.robot)

display.rate = -1
display.freq = 1
for i in range(5):
    print("displaying")
    display.displayFromSolver(mpc_generator.solver)
    time.sleep(1.0)

q0 [ 0.    -0.698  0.     1.571 -0.     0.698 -0.   ]
dict_keys(['link1', 'link2', 'link3', 'link4', 'link5', 'link6', 'link7'])
[[ 9.99999979e-01  1.37562903e-16 -2.03694277e-04  7.01775245e-01]
 [ 1.37598041e-16 -1.00000000e+00  1.72486120e-16 -1.13000000e-01]
 [-2.03694277e-04 -1.72514145e-16 -9.99999979e-01  5.25812805e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
sending to the robot!
u [ 2.26532121e+00  5.44202136e+01  3.30470088e+00 -2.43175077e+01
 -3.35165165e+00  3.85626619e+00 -1.01489342e-02]
x_des [ 0.    -0.698  0.     1.571 -0.     0.698 -0.     0.     0.     0.
  0.     0.     0.     0.   ]
iter    cost       merit      stop       grad       preg       dreg      step   ||ffeas||  ||gfeas||  ||hfeas||   dV-exp       dV      dPhi-exp     dPhi
   0  8.483e+02  0.000e+00  2.634e+03  4.920e+03  1.000e-09  1.000e-09  0.5000  1.583e+00  0.000e+00  0.000e+00  1.932e+03  1.908e+03  0.000e+00  0.000e+00
   1  3.181e+02  0.000e+00  8.229e+02  1.678e+03  

In [ ]:
# now we run it on an mpc loop. 




# we want
dx_des = [0, 0, -.5]
Tdes = np.copy(T0)
Tdes[:3, 3] += np.array(dx_des)

def get_error(x): 
    Tcurrent = flexiv_model.getInfo(x[:7],np.zeros(7))["poses"]["link7"]
    return  np.linalg.norm(pinocchio.log6( np.linalg.inv(Tdes) @ Tcurrent  ))

mpc_cfg  = Mpc_cfg()
mpc_cfg.dt = .1
mpc_cfg.gripperPose_run = 20

x0 = flexiv_model.q0

mpc_generator =  Mpc_generator(flexiv_model.robot.model, x0, Tdes,  mpc_cfg)


q = flexiv_model.q0
x0 = np.concatenate([q,np.zeros(7)])
x = x0


display = crocoddyl.MeshcatDisplay(flexiv_model.robot)

display.rate = -1
display.freq = 1


error = get_error(x0)
print("error ", error)

noise = 0
step = 0
trajectory = []
while error > 1e-2 and step < 100: 
    trajectory.append(np.copy(x))
    mpc_generator.update_x0(x)
    mpc_generator.solver.solve()
    # display.displayFromSolver(mpc_generator.solver)
    # time.sleep(0.01)
    u = mpc_generator.solver.us[0]
    x_des = mpc_generator.solver.xs[1]
    # print("x_0", mpc_generator.problem.x0)
    # print("x_0", mpc_generator.solver.xs[0])
    # print("x_des", x_des)
    # print("len ",  len(mpc_generator.solver.xs))
    x = x_des + noise * np.random.randn(14)

    error = get_error(x)
    print(f" step {step:03d} error is {error}")
    step +=1 
    


    


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7013/static/
error  0.4999999999999999
 step 000 error is 0.4709587572384229
 step 001 error is 0.43108518095953297
 step 002 error is 0.38947209857933146
 step 003 error is 0.349545468753122
 step 004 error is 0.31219354275820427
 step 005 error is 0.27753133068157604
 step 006 error is 0.24557970893889075
 step 007 error is 0.2163931952001204
 step 008 error is 0.19001424168533904
 step 009 error is 0.1664227046802668
 step 010 error is 0.14552128884083998
 step 011 error is 0.1271482494717002
 step 012 error is 0.11110038740217289
 step 013 error is 0.09715499320216106
 step 014 error is 0.0850862383676428
 step 015 error is 0.07467568109850188
 step 016 error is 0.06571824246897999
 step 017 error is 0.058025195309233855
 step 018 error is 0.05142534995479419
 step 019 error is 0.04576519560016409
 step 020 error is 0.04090843742557485
 step 021 error is 0.03673517606460333
 step 022 error is 0.033140869243